In [1]:
import sympy as sm
import sympy.physics.mechanics as me
me.init_vprinting(use_latex='mathjax')

# Section 1: Angular Velocity
Reference frames can be oriented relative to each other, using $R \in SO(3)$. 
If the relative orientation of the two reference frame changes w.r.t time, then the angular velocity ${}^A\bar{\omega}^B$, which reads the angular velocity of $\{B\}$ observed in $\{A\}$,
can be calculated.

If $\hat{b}_x$, $\hat{b}_y$, $\hat{b}_z$ are the right-handed mutually perpendicular unit vectors fixed in $\{B\}$,
then:

${}^A\bar{\omega}^B :=
\left(\frac{{}^A d\hat{b}_y}{dt} \cdot \hat{b}_z\right) \hat{b}_x +
\left(\frac{{}^A d\hat{b}_z}{dt} \cdot \hat{b}_x\right) \hat{b}_y +
\left(\frac{{}^A d\hat{b}_x}{dt} \cdot \hat{b}_y\right) \hat{b}_z $

The rotation matrix of $\{B\}$ w.r.t $\{A\}$ or (from A to B) ${{}^B R^A}$ is used to rotate the $SO(3)$ of $\{A\}$ to $\{B\}$:
i.e. $\hat{B} = {}^B R^A \hat{A}$

## The take-away:
__Given this ${}^B R^A$ at any instance of time and also the time derivatives of its each entries, we can compute ${}^A\bar{\omega}^B$. Computing $\frac{{}^A\hat{b}_x}{dt}$ requires to project the basis vectors in $\{B\}$ onto those of $\{A\}$. By doing so, the only the measure numbers is changing with time, not the basis vectors (as no longer expressed in $\{B\}$)__

$\begin{split}
\hat{b}_x & = c_{xx} \hat{a}_x + c_{xy} \hat{a}_y + c_{xz} \hat{a}_z \\
\hat{b}_y & = c_{yx} \hat{a}_x + c_{yy} \hat{a}_y + c_{yz} \hat{a}_z \\
\hat{b}_z & = c_{zx} \hat{a}_x + c_{zy} \hat{a}_y + c_{zz} \hat{a}_z\end{split}$

In [3]:
# create the dcm with time varying elements
# .dynamicsymbols() is implicit on t
cxx, cxy, cxz = me.dynamicsymbols("c_{xx}, c_{xy}, c_{xz}")
cyx, cyy, cyz = me.dynamicsymbols("c_{yx}, c_{yy}, c_{yz}")
czx, czy, czz = me.dynamicsymbols("c_{zx}, c_{zy}, c_{zz}")
B_R_A = sm.Matrix([[cxx, cxy, cxz],
                   [cyx, cyy, cyz],
                   [czx, czy, czz]]) 

In [8]:
A = me.ReferenceFrame('A')
B = me.ReferenceFrame('B')

# this reads orient B relative to A by B_R_A
B.orient_explicit(A, B_R_A.transpose()) # .orient_explicit() applies the orientation with transpose on default 
B.dcm(A)

⎡c_{xx}  c_{xy}  c_{xz}⎤
⎢                      ⎥
⎢c_{yx}  c_{yy}  c_{yz}⎥
⎢                      ⎥
⎣c_{zx}  c_{zy}  c_{zz}⎦

__Writing the unit vector $\hat{b}_x$, $\hat{b}_y$, $\hat{b}_z$ of $\{B\}$ in terms of $\hat{a}_x$, $\hat{a}_y$, $\hat{a}_z$ of $\{A\}$__

In [9]:
B.x.express(A)

c_{xx} a_x + c_{xy} a_y + c_{xz} a_z

In [10]:
B.y.express(A)

c_{yx} a_x + c_{yy} a_y + c_{yz} a_z

In [11]:
B.z.express(A)

c_{zx} a_x + c_{zy} a_y + c_{zz} a_z

To compute the angular velocities, the derivative of $\{B\}$'s unit vector, $\frac{{}^A\hat{b}_x}{dt}$, $\frac{{}^A\hat{b}_y}{dt}$, $\frac{{}^A\hat{b}_z}{dt}$ need to be calculated, and then the measure numbers can be obtained by dotting this derivative with the corresponding unit vector, such as $\frac{{}^A\hat{b}_x}{dt} \cdot \hat{b}_z$ 

In [17]:
# the derivative
B.x.express(A).dt(A)

c_{ẋx} a_x + c_{ẋy} a_y + c_{ẋz} a_z

Then each measure number of ${}^A\bar{\omega}^B$ are then:

In [19]:
# the measure numbers 
mnx = me.dot(B.x.express(A).dt(A), B.z)
mnx

c_{zx}⋅c_{ẋx} + c_{zy}⋅c_{ẋy} + c_{zz}⋅c_{ẋz}

In [20]:
mny = me.dot(B.y.express(A).dt(A), B.x)
mny

c_{xx}⋅c_{ẏx} + c_{xy}⋅c_{ẏy} + c_{xz}⋅c_{ẏz}

In [22]:
mnz = me.dot(B.z.express(A).dt(A), B.y)
mnz

c_{yx}⋅c_{żx} + c_{yy}⋅c_{ży} + c_{yz}⋅c_{żz}

In [24]:
A_w_B = mnx*B.x + mny*B.y + mnz*B.z
A_w_B

(c_{zx}⋅c_{ẋx} + c_{zy}⋅c_{ẋy} + c_{zz}⋅c_{ẋz}) b_x + (c_{xx}⋅c_{ẏx} + c_{xy}⋅c_{ẏy} + c_{xz}⋅c_{ẏz}) b_y + (c_{yx}⋅c_{żx} +
 c_{yy}⋅c_{ży} + c_{yz}⋅c_{żz}) b_z

__Exercise:__

At one instance of time, the dcm is:
$\begin{split}
{}^B\mathbf{C}^A =
    \begin{bmatrix}
      \frac{\sqrt{2}}{4}  & \frac{\sqrt{2}}{2}  & \frac{\sqrt{6}}{4} \\
      -\frac{\sqrt{3}}{2} &                  0  & \frac{1}{2} \\
      \frac{\sqrt{2}}{4}  & -\frac{\sqrt{2}}{2} & \frac{\sqrt{6}}{4}
    \end{bmatrix}
  \end{split}$

And the time derivative of the entries of the dcm are:
$
\begin{split}
\frac{d{}^B\mathbf{R}^A}{dt} =
    \begin{bmatrix}
      -\frac{\sqrt{6}}{2}-\frac{3 \sqrt{2}}{4} & -\frac{\sqrt{6}}{4}+\frac{3 \sqrt{2}}{2} &-\frac{3 \sqrt{6}}{4}+\sqrt{2}\\
      -1  &-\frac{1}{2} & - \sqrt{3}\\
      -\frac{\sqrt{6}}{2} + \frac{3 \sqrt{2}}{4} & - \frac{\sqrt{6}}{4} + \frac{3 \sqrt{2}}{2} & \frac{3 \sqrt{6}}{4}
    \end{bmatrix}
\end{split}
$
apply the definition of angular velocity to find ${}^A \bar{\omega}^B$

In [32]:
# the measure numbers of the angular velocity
B_R_A = sm.Matrix([
    [ sm.sqrt(2)/4,  sm.sqrt(2)/2, sm.sqrt(6)/4],
    [-sm.sqrt(3)/2,          0,       sm.S(1)/2],
    [ sm.sqrt(2)/4, -sm.sqrt(2)/2, sm.sqrt(6)/4]
])
B_R_A

⎡ √2    √2   √6 ⎤
⎢ ──    ──   ── ⎥
⎢ 4     2    4  ⎥
⎢               ⎥
⎢-√3            ⎥
⎢────   0    1/2⎥
⎢ 2             ⎥
⎢               ⎥
⎢ √2   -√2   √6 ⎥
⎢ ──   ────  ── ⎥
⎣ 4     2    4  ⎦

In [31]:
B_R_A_dt = sm.Matrix([
    [-sm.sqrt(6)/2 - 3*sm.sqrt(2)/4, -sm.sqrt(6)/4 + 3*sm.sqrt(2)/2, -3*sm.sqrt(6)/4 + sm.sqrt(2)],
    [                      -1,                     -sm.S(1)/2,               -sm.sqrt(3)],
    [-sm.sqrt(6)/2 + 3*sm.sqrt(2)/4, -sm.sqrt(6)/4 + 3*sm.sqrt(2)/2,            3*sm.sqrt(6)/4]
])
B_R_A_dt

⎡  √6   3⋅√2    √6   3⋅√2    3⋅√6     ⎤
⎢- ── - ────  - ── + ────  - ──── + √2⎥
⎢  2     4      4     2       4       ⎥
⎢                                     ⎥
⎢    -1          -1/2          -√3    ⎥
⎢                                     ⎥
⎢  √6   3⋅√2    √6   3⋅√2     3⋅√6    ⎥
⎢- ── + ────  - ── + ────     ────    ⎥
⎣  2     4      4     2        4      ⎦

In [42]:
# then compute the measure number by recognizing the pattern
mnx_ex = (B_R_A[2, :]*B_R_A_dt[1, :].transpose())[0, 0]
mnx_ex

-3⋅√2 
──────
  4   

In [45]:
mny_ex = (B_R_A[0, :]*B_R_A_dt[2, :].transpose())[0, 0]
mnz_ex = (B_R_A[1, :]*B_R_A_dt[0, :].transpose())[0, 0]

A_w_B = mnx_ex*B.x + mny_ex*B.y + mnz_ex*B.z
A_w_B

In [48]:
A_w_B.simplify()

-3⋅√2        ⎛    √3⎞       5⋅√2
────── b_x + ⎜3 - ──⎟ b_y + ──── b_z
  4          ⎝    2 ⎠        4

## A NOTE: how is $\frac{d {}^B R^A}{dt}$ or ${}^B\dot{R}^A$ the derivatives of the dcm obtained in the first place??

$\frac{dC}{dt}= \Omega C$, where $\Omega$ is the skew-symmetric representation of the angular velocity.

In practice (according to my experience so far), a time sequenced transformation $T \in SE(3)$ is generated by interpolation or time-optimal scaling, or by kinematics with constraints (convex programming). The $R \in SO(3)$ is included in $T$. The angular velocity is then computed from the time sequenced rotations. We can compute the angular velocity in $\{space\}$ or $\{body\}$ using:  

$\Omega_s = \dot{R}R^{-1}$ and

$\Omega_b = R^{-1}\dot{R}$ 

Then by setting $\Delta t = 1$, $\Omega_s = RR^{-1}$. Likewise for $\Omega_b$

# Section 3: Time Derivatives of Vectors

The time derivative of a unit vector fixed in $\{B\}$ is related to $\{B\}$'s angular velocity by:
$\frac{{}^Ad\hat{b}_x}{dt} = {}^A \bar{\omega}^B \times \hat{b}_x$. This indicates that the derivative is always normal to the unit vector because the magnitude of the unit vector is constant and the derivatives scales with the magnitude of the angular velocity:

$\frac{{}^Ad\hat{b}_x}{dt} = |{}^A \bar{\omega}^B| ({}^A \hat{\omega}^B \times \hat{b}_x)$
This reads the unit vector fixed in $\{B\}$ and observed in $\{A\}$.

__Case 1: fixed in $\{B\}$__

Now if vector $\bar{v} = v\hat{b}_x$ and v is constant w.r.t time then:
$\frac{{}^A d\bar{v}}{dt} = v({}^A \bar{\omega}^B \times \hat{b}_x) = {}^A \bar{\omega}^B \times v \hat{b}_x = {}^A \bar{\omega}^B \times \bar{v}$. Then we are able to extend this to express any vector fixed in $\{B\}$ and observed from $\{A\}$

<div class="alert alert-block alert-success">
<b>An intuitive understanding:</b> Think of this $\bar{u}$ represent a point in $\{b\}$, this can be any point on the rigid object, saying a basket ball, which is fixed in the basketball's body frame. What is calculated here, the $\frac{{}^A d\bar{v}}{dt}$ is the velocity of this point as you are standing still watching it passing by while it is also spinning. This measure of velocity is from your point of view. 
</div>

__Case 2: NOT fixed in $\{B\}$__

Now if the vector $\bar{u}$ is a vector not fixed in $\{B\}$, as previously, use the Product Rule. __We first need to express $\bar{u}$ in $\{B\}$:__ $\bar{u} = u_1\hat{b}_x + u_2\hat{b}_y + u_3\hat{b}_z$.

$\begin{split}
\frac{{}^Ad\bar{u}}{dt} &=
\dot{u}_1\hat{b}_x + \dot{u}_2\hat{b}_y + \dot{u}_3\hat{b}_z + u_1\frac{{}^Ad\hat{b}_x}{dt} + u_2\frac{{}^Ad\hat{b}_y}{dt} + u_3\frac{{}^Ad\hat{b}_z}{dt} \\
\frac{{}^Ad\bar{u}}{dt} &= 
\frac{{}^Bd\bar{u}}{dt} + u_1{}^A\bar{\omega}^B\times\hat{b}_x + u_2{}^A\bar{\omega}^B\times\hat{b}_y + u_3{}^A\bar{\omega}^B\times\hat{b}_z \\
\frac{{}^Ad\bar{u}}{dt} &= 
\frac{{}^Bd\bar{u}}{dt} + {}^A\bar{\omega}^B\times\bar{u}
\end{split}$

which reads: the time derivative of $\bar{u}$ which is moving in $\{B\}$ and observed in $\{A\}$ equals to the part (translation) in $\{B\}$ plus the part in $\{A\}$ (by rotation).

__This is powerful as it allows us to differentiate any vector if we know how it changes in a rotating reference frame relative to the reference frame from which we are observing the change!!!!__

<div class="alert alert-block alert-success">
<b>An intuitive understanding:</b> Think of this $\bar{u}$ represent a point in $\{b\}$, but the body is no longer a rigid body but a deformable one (it can also be other cases such as you throw a tube with a ball inside that can move within, here just one of the many illustrations), saying a pancake, which is not fixed in the pancake's body frame because it will be streched while spinning. What is calculated here, the $\frac{{}^A d\bar{v}}{dt}$ is the velocity of this point as you are standing still watching it passing by while it is also spinning with deformation. The point has relative motion in $\{B\}$. This measure of velocity from your point of view includes the pancake frame's motion plus the streching part of motion (vector + vector).
</div>

In [49]:
psi, theta, phi = me.dynamicsymbols("psi, theta, phi")

A = me.ReferenceFrame('A')
B = me.ReferenceFrame('B')
B.orient_body_fixed(A, (psi, theta, 0), 'ZXZ')

u1, u2, u3 = me.dynamicsymbols("u1, u2, u3")
u = u1*B.x + u2*B.y + u3*B.z
u

u₁ b_x + u₂ b_y + u₃ b_z

Now express $\bar{u}$ in $\{A\}$ and take the time derivative of the measure numbers to obtain $\frac{{}^Ad\bar{u}}{dt}$

In [50]:
u.express(A)

(u₁⋅cos(ψ) - u₂⋅sin(ψ)⋅cos(θ) + u₃⋅sin(ψ)⋅sin(θ)) a_x + (u₁⋅sin(ψ) + u₂⋅cos(ψ)⋅cos(θ) - u₃⋅sin(θ)⋅cos(ψ)) a_y + (u₂⋅sin(θ) + u₃⋅cos
(θ)) a_z

In [73]:
u.express(A).dt(A).simplify()

(-u₁⋅sin(ψ)⋅ψ̇ + u₂⋅sin(ψ)⋅sin(θ)⋅θ̇ - u₂⋅cos(ψ)⋅cos(θ)⋅ψ̇ + u₃⋅sin(ψ)⋅cos(θ)⋅θ̇ + u₃⋅sin(θ)⋅cos(ψ)⋅ψ̇ + sin(ψ)⋅sin(θ)⋅u₃̇ - sin(ψ)
⋅cos(θ)⋅u₂̇ + cos(ψ)⋅u₁̇) a_x + (u₁⋅cos(ψ)⋅ψ̇ - u₂⋅sin(ψ)⋅cos(θ)⋅ψ̇ - u₂⋅sin(θ)⋅cos(ψ)⋅θ̇ + u₃⋅sin(ψ)⋅sin(θ)⋅ψ̇ - u₃⋅cos(ψ)⋅cos(θ)⋅
θ̇ + sin(ψ)⋅u₁̇ - sin(θ)⋅cos(ψ)⋅u₃̇ + cos(ψ)⋅cos(θ)⋅u₂̇) a_y + (u₂⋅cos(θ)⋅θ̇ - u₃⋅sin(θ)⋅θ̇ + sin(θ)⋅u₂̇ + cos(θ)⋅u₃̇) a_z

If using the expression $\frac{{}^Ad\bar{u}}{dt} = \frac{{}^Bd\bar{u}}{dt} + {}^A\bar{\omega}^B\times\bar{u}$, there is no need to express $\bar{u}$ in $\{A\}$.

$\frac{{}^Bd\bar{u}}{dt}$:

In [53]:
u.dt(B)

u₁̇ b_x + u₂̇ b_y + u₃̇ b_z

then ${}^A \bar{\omega}^B$ 

In [56]:
A_w_B = B.ang_vel_in(A)
A_w_B

θ̇ b_x + sin(θ)⋅ψ̇ b_y + cos(θ)⋅ψ̇ b_z

In [57]:
u.dt(B) + me.cross(A_w_B, u)

(-u₂⋅cos(θ)⋅ψ̇ + u₃⋅sin(θ)⋅ψ̇ + u₁̇) b_x + (u₁⋅cos(θ)⋅ψ̇ - u₃⋅θ̇ + u₂̇) b_y + (-u₁⋅sin(θ)⋅ψ̇ + u₂⋅θ̇ + u₃̇) b_z

In [59]:
u.express(A).dt(A).express(B).simplify()

(-u₂⋅cos(θ)⋅ψ̇ + u₃⋅sin(θ)⋅ψ̇ + u₁̇) b_x + (u₁⋅cos(θ)⋅ψ̇ - u₃⋅θ̇ + u₂̇) b_y + (-u₁⋅sin(θ)⋅ψ̇ + u₂⋅θ̇ + u₃̇) b_z

In [65]:
# this shows .dt() applied the expression with cross-product internally
u.dt(A)

(-u₂⋅cos(θ)⋅ψ̇ + u₃⋅sin(θ)⋅ψ̇ + u₁̇) b_x + (u₁⋅cos(θ)⋅ψ̇ - u₃⋅θ̇ + u₂̇) b_y + (-u₁⋅sin(θ)⋅ψ̇ + u₂⋅θ̇ + u₃̇) b_z

# Section 4: Angular Acceleration 
The angular acceleration of B when observed from A is: ${}^A\alpha^B := \frac{{}^Ad}{dt} {}^A\bar{\omega}^B$. Since ${}^A\bar{\omega}^B$ is a vector, it can be differentiated w.r.t A:

$\begin{split}\frac{{}^Ad}{dt} {}^A\bar{\omega}^B & =
\frac{{}^Bd}{dt} {}^A\bar{\omega}^B + {}^A\bar{\omega}^B \times {}^A\bar{\omega}^B \end{split}$ which gives:

$\frac{{}^Ad}{dt} {}^A\bar{\omega}^B = \frac{{}^Bd}{dt} {}^A\bar{\omega}^B$, this means the change of velocity is regardless of the frame from which it is observed.

Seccussive addition of accelerations does not result in a valid total angular acceleration: ${}^A\bar{\alpha}^Z \neq {}^A\bar{\alpha}^B + {}^B\bar{\alpha}^C + \ldots + {}^Y\bar{\alpha}^Z$

Angular accelerations derived from successive orientations require an explicit differentiation of the associated angular velocity vector. There unfortunately is no theorem that simplifies this calculation as we see with orientation and angular velocity.